# Modèles multimodaux - Voting

## README
Ce notebook permet la création et l'évaluation de modèles de voting.

Il réalise tout d'abord certaines opérations préalables (chapitre 1), dont la définition des variables globales d'exécution (**A METTRE A JOUR LORS D'UNE PREMIERE UTILISATION**)

Le chapitre 2 crée et évalue un modèle de voting simple, par moyennage des prédictions de 2 modèles (1 images et 1 texte)

Le chapitre 3 propose une version avancée, avec pondérations, apprises ou non, des résultats des deux modèles utilisés

## 1. Préparation

In [ ]:
import sys
from pathlib import Path

project_root = Path().resolve().parent
if not project_root in [Path(p).resolve() for p in sys.path]:
    sys.path.append(str(project_root))

from src import PATHS

In [ ]:
import os
import time
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from src.models.multimodal import MultiModalVoter, MultiModalClassWeightedVoter, MultiModalLogisticRegressor
from src.visualization.visualize import visual_classification_report

## 2. Chargement des données

In [ ]:
text_features = pd.read_parquet(os.path.join(PATHS.processed_data, 'df_txt_ocr1.parquet'))
text_features.shape

In [ ]:
image_features = pd.read_parquet(os.path.join(PATHS.processed_data, "df_img_features_flattened.parquet"))
image_features.shape

In [ ]:
features = text_features.join(image_features, how="inner")
del image_features, text_features
features.shape

In [ ]:
data_sets = pd.read_parquet(os.path.join(PATHS.metadata, "df_data_sets.parquet"))
labels = pd.read_parquet(os.path.join(PATHS.metadata, "df_encoded_labels.parquet"))

In [ ]:
# au cas où features soit incomplet
data_sets = data_sets.join(features[[]], how="inner")
labels = labels.join(features[[]], how="inner")

features.shape, data_sets.shape, labels.shape

In [ ]:
X_train = features[data_sets.data_set == "train"]
y_train = labels[data_sets.data_set == "train"]

X_val = features[data_sets.data_set == "val"]
y_val = labels[data_sets.data_set == "val"]

X_test = features[data_sets.data_set == "test"]
y_test = labels[data_sets.data_set == "test"]

del features, labels, data_sets

## 3. Chargement des pipelines

In [ ]:
available_pipelines = [pipeline for pipeline in  os.listdir(PATHS.pipelines) if pipeline.endswith(".pkl")]
img_pipelines = [pipeline for pipeline in available_pipelines if pipeline.startswith("img")]
txt_pipelines = [pipeline for pipeline in available_pipelines if pipeline.startswith("txt")]
print(img_pipelines)
print(txt_pipelines)

## TODO: mettre dans src 

In [ ]:
def df_to_serie(df):
    assert len(df.columns) == 1
    return df[df.columns[0]]

In [ ]:
with open(os.path.join(PATHS.pipelines, "img_lgbm.pkl"), "rb") as f:
    img_pipeline = pickle.load(f)
with open(os.path.join(PATHS.pipelines, "txt_ml.pkl"), "rb") as f:
    txt_pipeline = pickle.load(f)


## 4. Chargement des modèles

In [ ]:
available_models = [model for model in  os.listdir(PATHS.models) if model.endswith(".pkl")]
img_models = [model for model in available_models if model.startswith("img")]
txt_models = [model for model in available_models if model.startswith("txt")]
print(img_models)
print(txt_models)

In [ ]:
with open(os.path.join(PATHS.models, "img_lgbm.pkl"), "rb") as f:
    img_model = pickle.load(f)
with open(os.path.join(PATHS.models, "txt_logistic_regression.pkl"), "rb") as f:
    txt_model = pickle.load(f)

In [ ]:
img_model, txt_model

## 5. Voting simple

### 5.1. Averaging 

In [ ]:
multimodal_voter = MultiModalVoter(img_model, img_pipeline, txt_model, txt_pipeline) 
visual_classification_report(multimodal_voter, X_test, y_test, "Averaging voter", compare_with_components=True)

### 5.2. Max value

In [ ]:
multimodal_voter = MultiModalVoter(img_model, img_pipeline, txt_model, txt_pipeline, method = "max") 
visual_classification_report(multimodal_voter, X_test, y_test, "Max value voter", compare_with_components=True)

### 5.3. Weighted
Nous allons utiliser le jeu de validation pour déterminer la meilleure répartition des poids entre les modèles image et texte.

In [ ]:

accuracies = []
for alpha in tqdm(np.linspace(0,1, 101)):
    multimodal_voter = MultiModalVoter(img_model, img_pipeline, txt_model, txt_pipeline, method="weighted", weights = (alpha, 1-alpha))
    accuracies.append([alpha, accuracy_score(y_val, multimodal_voter.predict(X_val))])

In [ ]:
plt.figure(figsize=(8,8))
plt.plot((0,0.49),(0.8421, 0.842), 'r--', lw=1)
plt.plot((0.49,0.49),(0.55, 0.842), 'r--', lw=1)
plt.plot(
    tuple(r[0] for r in accuracies),
    tuple(r[1] for r in accuracies)
)

plt.xticks(list(plt.xticks()[0]) + [0.49])
plt.yticks(list(plt.yticks()[0]) + [0.842])

# Colorer le tick x = 0.49 en rouge
for label in plt.gca().get_xticklabels():
    if label.get_text() == '0.49':
        label.set_color('red')

# Colorer le tick y = 0.8421 en rouge
for label in plt.gca().get_yticklabels():
    if label.get_text() == '0.842':
        label.set_color('red')
plt.xlim(0, 1)
plt.ylim(0.55, 0.87)
plt.grid()
plt.xlabel("Ratio\n(0 = text only // 1 = image only)")
plt.ylabel("Exactitude (données de validation)")
plt.title("Weighted voter - Exactitude en fonction du poids texte/image")

In [ ]:
print("ratio |accuracy")
print("------+--------")
for al, ac in accuracies[40:60]:
    print(f"{al:.2f}  | {100*ac:.2f}%")

In [ ]:
multimodal_voter = MultiModalVoter(img_model, img_pipeline, txt_model, txt_pipeline, method="weighted", weights = (0.49, 0.51))
visual_classification_report(multimodal_voter, X_test, y_test, "0.49-weighted Voter", compare_with_components=True)

## 6. Pondération par classe

In [ ]:
multimodal_voter = MultiModalClassWeightedVoter(img_model, img_pipeline, txt_model, txt_pipeline)
multimodal_voter.fit(X_val, y_val)

In [ ]:
visual_classification_report(multimodal_voter, X_test, y_test, "Class-Weighted voter", compare_with_comp@bonents=True)

In [ ]:
max(txt_weights), min(txt_weights)
print(" class | txt weight | img weight ")
print("-------+------------+------------")
for c, (t, i) in enumerate(zip(txt_weights, img_weights)):
    print(f"   {c:02d}  |    {t:.2f}    |    {i:.2f}")

In [ ]:
txt_weights = (multimodal_voter.txt_weights/multimodal_voter.weights_sum).flatten()
img_weights = (multimodal_voter.img_weights/multimodal_voter.weights_sum).flatten()

indices = np.arange(len(txt_weights))

plt.bar(indices, np.ones(16), alpha = 0.75) # pour "sauter la couleur bleue et retrouver les couleurs des graphes précédents
plt.bar(indices, img_weights, bottom=txt_weights, label='image model', alpha = 0.75)
plt.bar(indices, txt_weights, label='text model', alpha = 0.75)

plt.ylabel('Poids')
plt.xlabel('Classe')
plt.title('Valeurs des poids par classe associés aux modèles constitutifs')
plt.xticks(indices)
plt.ylim(0, 1.05)
plt.legend(loc='lower right')
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()
